In [1]:
import torch
from math import floor, ceil
    
def warp_matrix(x):
    a=x
    b = torch.cumsum(a,0)
    t_dim = [int(ceil(b[-1])),a.shape[0]]
    trans_mat = torch.zeros(t_dim)
    
    prev_ind =[0,0]
    for i, x in enumerate(zip(a,b)):
        ai, bi = x
        this_ind = [floor(bi),i]
        if this_ind[0] == prev_ind[0]:
            trans_mat[this_ind[0], this_ind[1]] = ai
        else: # we just crossed an integer boundary
            tmp = bi - floor(bi)
            trans_mat[this_ind[0], this_ind[1]] = tmp
            trans_mat[this_ind[0]-1, this_ind[1]] = ai -tmp
        prev_ind = this_ind

    assert( (a - trans_mat.sum(0)).abs().max() < 1e-6)
    assert((torch.ones(trans_mat.shape[0] -1) -trans_mat.sum(1)[:-1]).abs().max() <1e-6)

    return trans_mat

class WarpMatrix(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return warp_matrix(x)
    
a = torch.FloatTensor(10).uniform_()
print(a)  
trans_mat = warp_matrix(a)
print(trans_mat.sum(1))
#b = torch.autograd.Variable([a.shape[0], a.shape[0]])


 0.2290
 0.4237
 0.0629
 0.0395
 0.7125
 0.4943
 0.7146
 0.4915
 0.4065
 0.8245
[torch.FloatTensor of size 10]


 1.0000
 1.0000
 1.0000
 1.0000
 0.3990
[torch.FloatTensor of size 5]



In [3]:
!export CUDA_HOME="/usr/local/cuda"
from warpctc_pytorch import CTCLoss


ImportError: libwarpctc.so: cannot open shared object file: No such file or directory

In [39]:
from torch.autograd import Variable
class FittedWarp(torch.nn.Module):
    def __init__(self, w_shape):
        super().__init__()
        self.w = torch.nn.Parameter(torch.randn(w_shape))

    def forward(self, x):
        tmp1 = x @ self.w
        tmp = torch.nn.Sigmoid()(tmp1)
        trans_mat = warp_matrix(tmp)
        print(tmp, trans_mat)
        return trans_mat @ x
    
dim = 6
a =Variable(torch.randn([10, dim]))
warp = FittedWarp(dim)
y = warp.forward(a)
print(y)

TypeError: must be real number, not Variable